In [14]:
using Rocket
using Distributions
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

In [15]:
function createSubgraph(index::Int)
    noise_add = AdditionNode()
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    yn    = datavar(:yn, Float64)
    
    connect!(noise_add, :in2, noise, 1)
    connect!(noise_add, :out, yn, 1)
    
    return (yn, noise_add)
end

function createGraph(size::Int)    
    c0_add = AdditionNode()
    
    c0       = constvar(:c0, 1.0)
    x0_prior = datavar(:x0_prior, Normal{Float64})
    
    connect!(c0_add, :in1, x0_prior, 1)
    connect!(c0_add, :in2, c0, 1)
    
    index = 1
    
    prev_c_add = c0_add
    
    xs = Vector{RandomVariable}(undef, size)
    ys = Vector{DataVariable}(undef, size)
    
    while index < size
        yn, noise_add = createSubgraph(index)
        xn            = randomvar(:xn, 3)
        
        c_add = AdditionNode()
        cn    = constvar(:cn, 1.0)
        
        connect!(prev_c_add, :out, xn, 1)
        connect!(noise_add, :in1, xn, 2)
        connect!(c_add, :in1, xn, 3)
        connect!(c_add, :in2, cn, 1)
        
        activate!(prev_c_add)
        activate!(noise_add)
        
        xs[index] = xn
        ys[index] = yn
        
        prev_c_add = c_add
        
        index += 1
    end
    
    last_noise_add = AdditionNode();
    
    x_last     = randomvar(:x_last, 2)
    y_last     = datavar(:ylast, Float64)
    last_noise = constvar(:last_noise, Normal(0.0, sqrt(200.0)))
    
    connect!(prev_c_add, :out, x_last, 1)
    connect!(last_noise_add, :in1, x_last, 2)
    connect!(last_noise_add, :in2, last_noise, 1)
    connect!(last_noise_add, :out, y_last, 1)
    
    activate!(prev_c_add)
    activate!(last_noise_add)
    
    xs[size] = x_last
    ys[size] = y_last
    
    return (xs, ys, x0_prior)
end

createGraph (generic function with 1 method)

In [29]:
function smoothing(data)
    N = length(data)
    
    xs, ys, x_prior = createGraph(N);
    
    messages      = Vector{Normal{Float64}}(undef, N)
    subscriptions = Vector{Teardown}(undef, N)
    
    @inbounds for (index, x) in enumerate(xs)
        subscriptions[index] = subscribe!(belief(x), (d) -> messages[index] = getdata(d))
    end
    
    # subscribe!(inference(xs[1]) |> take(1), (d) -> messages[1] = d)
    # subscribe!(inference(xs[N]) |> take(1), (d) -> messages[N] = d)
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    for i in 1:N
       update!(ys[i], data[i])
    end
    
#     for i in 2:N-1
#         subscribe!(inference(xs[i]) |> take(1), (d) -> messages[i] = d)
#     end
    
    foreach(unsubscribe!, subscriptions)
    
    return messages
end

smoothing (generic function with 1 method)

In [33]:
N = 5
data = collect(1:N) + sqrt(200.0) * randn(N);

In [34]:
@time smoothing(data)

 15.332653 seconds (29.34 M allocations: 1.554 GiB, 3.56% gc time)


5-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=6.990703165263703, σ=6.311944030978032)
 Normal{Float64}(μ=7.990703165263702, σ=6.311944030978031)
 Normal{Float64}(μ=8.990703165263703, σ=6.311944030978033)
 Normal{Float64}(μ=9.990703165263703, σ=6.311944030978033)
 Normal{Float64}(μ=10.990703165263705, σ=6.311944030978033)

In [8]:
@benchmark smoothing($data)

BenchmarkTools.Trial: 
  memory estimate:  15.29 MiB
  allocs estimate:  401712
  --------------
  minimum time:     32.107 ms (0.00% GC)
  median time:      33.409 ms (0.00% GC)
  mean time:        36.060 ms (7.79% GC)
  maximum time:     44.976 ms (23.93% GC)
  --------------
  samples:          139
  evals/sample:     1

In [5]:
@time smoothing(data)

  5.132938 seconds (15.65 M allocations: 801.089 MiB, 3.49% gc time)


600-element Array{StochasticMessage{Normal{Float64}},1}:
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=0.046967100321060204, σ=0.5772540681943419))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=1.0469671003210603, σ=0.577254068194342))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=2.0469671003210603, σ=0.577254068194342))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=3.0469671003210603, σ=0.5772540681943419))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=4.046967100321059, σ=0.5772540681943418))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=5.04696710032106, σ=0.5772540681943419))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=6.046967100321059, σ=0.5772540681943419))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=7.04696710032106, σ=0.577254068194342))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=8.04696710032106, σ=0.577254068194342))
 StochasticMessage{Normal{Float64}}(Normal{Float64}(μ=9.04696710032106, σ=0.5772

In [9]:
?SizedArray

search:

Couldn't find SizedArray
Perhaps you meant StridedArray or SubArray


No documentation found.

Binding `SizedArray` does not exist.
